In [ ]:
import os
# Get the current working directory
current_path = os.getcwd()

# Define the relative paths to your data and .venv folders
data_path = os.path.join(current_path, 'data')
venv_path = os.path.join(current_path, '.venv')

# Use the relative paths in your code
OPENSLIDE_PATH = os.path.join(venv_path, 'Lib', 'site-packages', 'openslide-win64-20230414', 'bin')

if hasattr(os, 'add_dll_directory'):
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide
import utils.preprocessing as pp
from openslide import open_slide
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
slide_path = os.path.join(data_path, 'training', 'tumor', 'tumor_001.tif')
ann_path = os.path.join(data_path, 'training', 'lesion_annotations_training',
                         'tumor_001.xml')

In [ ]:
slide = open_slide(slide_path)
f_mask = pp.foreground_background_segmentation(slide_path)
pp.visualize_mask_on_slide_side_by_side(slide, f_mask, level=3)

In [ ]:
polygons = pp.annotations_to_coordinates(ann_path)
p_samples, p_sample_origins = pp.sample_positive_patches(slide_path, polygons, 256, 10)

In [ ]:
gt_mask = pp.coordinates_to_mask(polygons, slide.dimensions)
p_sample_labels = pp.assign_patch_labels(256, p_sample_origins, gt_mask)

In [ ]:
pp.visualise_patch_and_label(p_samples[0], p_sample_labels[0])


In [ ]:
n_samples, n_sample_origins = pp.sample_negative_patches(slide_path, f_mask, polygons, 256, 10)
n_sample_labels = pp.assign_patch_labels(256, n_sample_origins, gt_mask)
pp.visualise_patch_and_label(n_samples[0], n_sample_labels[0])

In [ ]:
bc_samples, bc_sample_origins = pp.boundary_centered_sample_patches(slide_path, polygons, 256, 10)
bc_sample_labels = pp.assign_patch_labels(256, bc_sample_origins, gt_mask)
pp.visualise_patch_and_label(bc_samples[0], bc_sample_labels[0])

In [ ]:
b_samples, b_sample_origins = pp.sample_boundary_patches(slide_path, polygons, 256, 10)
b_sample_labels = pp.assign_patch_labels(256, b_sample_origins, gt_mask)
pp.visualise_patch_and_label(b_samples[0], b_sample_labels[0])